# **0.Librairies**

In [1]:
# Data manipulation
import numpy as np
import pandas as pd

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from scipy.stats import ks_2samp
# Modèles
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
# Grid Search
from sklearn.model_selection import GridSearchCV
# Cross Validation
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.metrics import make_scorer, f1_score

# Imbalanced-learn
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# XGBoost et LightGBM
import xgboost as xgb
import lightgbm as lgb

# PyTorch 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# UCI ML Repo
from ucimlrepo import fetch_ucirepo

# **I. Formulation du problème**

### **1. Définition du problème**

Nous allons utiliser le dataset [Online Shoppers Purchasing Intention](https://archive.ics.uci.edu/dataset/468/online+shoppers+purchasing+intention+dataset) pour prédire si une session d'un utilisateur aboutira à un achat ou non. Il s'agit donc d'un problème de **classification binaire**.

- **Objectif :** Construire un modèle prédictif capable de classer les sessions en deux catégories :
    - **Revenue = True** : la session se termine par un achat.
    - **Revenue = False** : la session ne se termine pas par un achat.

### **2. Formulation mathématique**

Soit :
- Un ensemble de données $X=\{x_1,x_2,...,x_n\}$ où chaque xi est un vecteur de caractéristiques représentant une session. 
- Une variable cible $Y=\{y_1, y_2, ..., y_n\}$ où $y_i \in \{0, 1\}$ (0 pour "pas d'achat", 1 pour "achat").

Le but est de trouver une fonction $f : X \rightarrow Y$ telle que pour un nouvel exemple $x, f(x)$ prédit correctement $y$.

# **II. Exploration du dataset**

*Informations according to the official documentation:*

The dataset consists of feature vectors belonging to 12,330 sessions. 

The dataset was formed so that each session would belong to a different user in a 1-year period to avoid any tendency to a specific campaign, special day, user profile, or period.

It cointains no missing value.

In [2]:
# Importation des données 
online_shoppers_purchasing_intention_dataset = fetch_ucirepo(id=468) 
  
# Récupérations des variables
variables = online_shoppers_purchasing_intention_dataset.variables 
X = online_shoppers_purchasing_intention_dataset.data.features 
y = online_shoppers_purchasing_intention_dataset.data.targets 

In [3]:
# Sauvegarde des données
variables.to_csv('../data/online_shoppers_purchasing_intention_dataset.csv', index=False)
X.to_csv('../data/features.csv', index=False)
y.to_csv('../data/targets.csv', index=False)

# Chargement des données
variables = pd.read_csv('../data/online_shoppers_purchasing_intention_dataset.csv')
X = pd.read_csv('../data/features.csv')
y = pd.read_csv('../data/targets.csv')

## **1. Analyse exploratoire des données**

Cette partie est dédiée à la **compréhension générale des variables** et consistera en l'étude des:
- **Variables numériques :** Analyse des distributions (moyenne, médiane, écart-type).
- **Variables catégorielles :** Comptage des occurrences, encodage.

### **a. Variables Dataset**

In [ ]:
variables.info()

In [5]:
# On supproime les variables vides
variables = variables[['name', 'role', 'type', 'missing_values']]

In [ ]:
variables

### **b. Target: Revenue (y)**

In [ ]:
y.info()

In [ ]:
y.describe()

In [ ]:
print(y['Revenue'].sum()/len(y))

# On affiche la distribution de la variable cible
sns.countplot(x='Revenue', data=y)
plt.show()

On observe que seulement **14,5%** des utilisateurs ont acheté.

### **c. Features (X)**

#### Informations générales

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
X.describe()

On remarque en effet qu'il n'y a aucune valeur manquante.

## **2. Visualisations initiales**

### **a. Visualisation des features numériques**

In [ ]:
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
n = len(numeric_features)
cols = 4
rows = n // cols + (n % cols > 0)

# Créer la figure et les sous-graphes
fig, axes = plt.subplots(rows, cols, figsize=(16, 12))
axes = axes.flatten()
for i, feature in enumerate(numeric_features):
    sns.histplot(X[feature], kde=True, ax=axes[i])
    axes[i].set_title(f'Distribution de {feature}')
for i in range(n, len(axes)):
    fig.delaxes(axes[i])
plt.tight_layout()
plt.show()

- La majorité des distributions ont une forte concentration de données **proches de zéro**, particulièrement pour les interactions avec des pages spécifiques du site (pages administratives, informatives et produits). On peut supposer que que seulement une petite fraction des utilisateurs s'engagent sur ce types de pages.

- L'analyse des taux de sortie et de rebond pourrait aider à **identifier des problèmes d'engagement**, notamment si certains types de pages (comme les pages de produits ou informatives) ont des taux de sortie plus élevés.

In [ ]:
categorical_features = X.select_dtypes(include=['object', 'bool']).columns.tolist()
n = len(categorical_features)
cols = 3

# Créer la figure et les sous-graphes
fig, axes = plt.subplots(1, cols, figsize=(16, 4))
axes = axes.flatten()
for i, feature in enumerate(categorical_features):
    sns.histplot(X[feature], kde=True, ax=axes[i])
    axes[i].set_title(f'Distribution de {feature}')
for i in range(n, len(axes)):
    fig.delaxes(axes[i])
plt.tight_layout()
plt.show()

On remarque plusieurs choses:
- Un comportement de **saisonalité**, avec des pics au printemps et en automne.
  
- Une présence plus importante de **visiteurs réccurents**, cela indique que le site a un bon taux de fidélisation.
  
- Les achats semblent assez **équitablement répartis** au cours de la semaine, on retrouve une proportion cohérente d'environ 5/7 en semaine et 2/7 le week-end. Cela pourrait indiquer que le site est autant utilisé pour des activités professionnelles que récréatives.

Aucune variable nulle dans les **features** et le **target** dataset, nous pouvons donc directement travailler sur les données.

#### **b. Détection de corrélations :**
- **Matrice de corrélation** pour identifier les variables fortement liées.

In [ ]:
plt.figure(figsize=(12, 10))
correlation = X[numeric_features].corr()
sns.heatmap(correlation, annot=True, cmap='coolwarm')
plt.title('Matrice de corrélation')
plt.show()

**Corrélations fortes :**

- **ProductRelated** et **ProductRelated_Duration** (0.86) : Il est logique que le nombre de pages de produits visitées soit fortement corrélé avec la durée passée sur ces pages. Plus les utilisateurs visitent de pages produits, plus ils passent de temps sur ces pages.

- **BounceRates** et **ExitRates** (0.91) : Ces deux variables sont également très corrélées. Cela indique que les sessions avec un taux de rebond élevé ont tendance à se terminer rapidement (forte probabilité de quitter le site après avoir consulté peu de pages).

**Corrélations modérées :**

- **Informational** et **Informational_Duration** (0.62) : De même, une corrélation assez élevée entre le nombre de pages informatives visitées et la durée passée sur ces pages, ce qui est également intuitif.

- **Administrative** et **Administrative_Duration** (0.6) : La relation entre le nombre de pages administratives visitées et la durée passée sur ces pages est modérée. Plus les utilisateurs interagissent avec des pages administratives, plus ils passent de temps sur ces pages.

Nous allons utiliser `sns.violinplot` pour observer les distributions de chaque **feature** contre le **target**.

In [ ]:
data = pd.concat([X, y], axis=1)

plt.figure(figsize=(16, 12))
for i, feature in enumerate(numeric_features, 1):
    plt.subplot(4, 4, i)
    sns.violinplot(x='Revenue', y=feature, data=data, split=True)
    plt.title(f'{feature} vs Revenue')

plt.tight_layout()
plt.show()

- **Pages produits** : Les variables liées aux pages produits (*ProductRelated* et *ProductRelated_Duration*) sont les meilleures variables pour différencier les utilisateurs générant des revenus de ceux qui n'en génèrent pas. Plus un utilisateur interagit avec les pages produits, plus il est susceptible de générer un revenu.

- **Taux de rebond** et **taux de sortie** : Les utilisateurs qui génèrent des revenus tendent à avoir des taux de rebond et taux de sortie plus faibles, ce qui indique un engagement plus profond avec le site.

- **PageValues** : C’est l'une des variables les plus discriminantes, les utilisateurs générant des revenus ayant des valeurs de page beaucoup plus élevées.

- Les autres facteurs n'ont pas un impact significatif sur la génération de revenus, du moins dans cette visualisation.

# **III. Modélisation**

## **1. Prétraitement des données**

### **a. Encodage des variables catégorielles**

Convertir les variables binaires en entiers :

In [17]:
data['Weekend'] = data['Weekend'].astype(int)
data['Revenue'] = data['Revenue'].astype(int)


One-Hot Encoding pour les variables catégorielles avec plus de deux catégories :

In [18]:
data = pd.get_dummies(data, columns=['Month', 'VisitorType'])

In [ ]:
data.head()

### **b. Normalisation ou standardisation des variables numériques**
Standardisation pour uniformiser l'échelle des variables :

In [20]:
scaler = StandardScaler()
data[numeric_features] = scaler.fit_transform(data[numeric_features])

### **c. Gestion du déséquilibre des classes**
SMOTE pour sur-échantillonner la classe minoritaire :

In [21]:
X = data.drop('Revenue', axis=1)
y = data['Revenue']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

### **d. Division des données en ensembles d'entraînement et de test**
Séparation les données en ensembles d'entraînement et de test :

In [22]:
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, 
    y_resampled, 
    test_size=0.3, # 30% des données pour le test
    random_state=42, # Fixer le random_state pour obtenir les mêmes résultats
    stratify=y_resampled # Stratification pour conserver la distribution de la variable cible
    )

## **2. Algorithmes de Classification**

Nous allons utliser les méthodes suivantes classiques puis comparer leurs performances:
- Régression Logistique
- Arbre de Décision
- Forêt Aléatoire
- XGBoost
- Réseau de Neurones avec PyTorch

### **a. Implémentation des modèles**

#### Régression Logistique

In [23]:
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

#### Arbre de Décision

In [24]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

#### Forêt Aléatoire

In [25]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

#### XGBoost

In [26]:
xgb_model = xgb.XGBClassifier(eval_metric='logloss')
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

#### RN avec PyTorch

In [27]:
# Convertir les données en tenseurs
X_train_tensor = torch.tensor(X_train.values.astype(float), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values.astype(float), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Créer des DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [28]:
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.layer2 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 2)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.relu(self.layer1(x))
        out = self.relu(self.layer2(out))
        out = self.output(out)
        return out

model = NeuralNet(input_size=X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    all_preds = []
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_preds.extend(predicted.numpy())
    
    print(f'Accuracy du réseau sur les données de test : {100 * correct / total}%')

### **b. Évaluation des modèles**

La précision seule perd en fiabilité dans un scénario où les classes sont déséquilibrées. Nous allons donc utiliser 3 méthodes d'évaluation qui sont particulirement pertinentes dans ce cas, et importées de `sklearn.metrics`: 

- `classification_report`:
  - la **précision** du modèle (proportion des prédictions positives qui sont correctes).
  - le **recall** (proportion des vrais positifs qui sont correctement prédits).
  - le **f1-score** (moyenne harmonique de la précision et du rappel).

- `confusion_matrix`: affiche au sein d'une matrice 2x2 les **True Positives**, **True Negatives**, **False Positives** et **False Negatives**.

- `roc_auc_score`: **ROC** (Receiver Operating Characteristic) est le tracé de **True Positive Ratio** contre **False Positive Ratio** pour chaque seuil de classification possible entre 0 et 1 (dans la pratique, à des intervalles choisis). L'**AUC** est l'aire sous la courbe ROC, plus il est proche de 1, plus le modèle est performant, et s'il est proche de 0.5, cela équivaut à une classification aléatoire.

In [31]:
predictions = {
    'Régression Logistique': y_pred_lr,
    'Arbre de Décision': y_pred_dt,
    'Forêt Aléatoire': y_pred_rf,
    'XGBoost': y_pred_xgb,
    'Réseau de Neurones (PyTorch)': all_preds
}

#### **Classification Report**

In [ ]:
results = []

for model_name, y_pred in predictions.items():
    # Récupération des informations du rapport de classification sous forme de dictionnaire
    report = classification_report(y_test, y_pred, output_dict=True)
    precision_0 = report['0']['precision']
    precision_1 = report['1']['precision']
    recall_0 = report['0']['recall']
    recall_1 = report['1']['recall']
    f1_0 = report['0']['f1-score']
    f1_1 = report['1']['f1-score']

    results.append({
        'Modèle': model_name,
        'Précision False': precision_0,
        'Précision True': precision_1,
        'Recall False': recall_0,
        'Recall True': recall_1,
        'F1-Score False': f1_0,
        'F1-Score True': f1_1
    })

df_results = pd.DataFrame(results)
df_results

La **forêt aléatoire** semble obtenir les meilleurs résultats.

#### **Confusion Matrix**

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for i, (model_name, y_pred) in enumerate(predictions.items()):
    ax = axes[i//3, i%3] 
    cm = confusion_matrix(y_test, y_pred)  # Calcul de la matrice de confusion
    sns.heatmap(cm, annot=True, fmt='d', cmap='plasma', ax=ax)
    ax.set_title(f'Matrice de Confusion - {model_name}')
    ax.set_xlabel('Prédictions')
    ax.set_ylabel('Vérités terrain')

axes[1][2].axis('off')
plt.tight_layout()
plt.show()


La **forêt aléatoire** et **XGBoost** semblent légèrement meilleurs en termes de réduction des faux positifs et faux négatifs.

#### **ROC-AUC Score**

Afin de tracer la courbe **ROC** à différents seuils de classification, on calcule les probabilités d'appartenance à chaque classe pour chaque observation. En pratique, seule la probabilité $P(classe=1)$ nous intéresse. Pour obtenir ces probabilités, nous pouvons utiliser la méthode `predict_proba` de `sklearn`. Cependant, notre **RN** ne dispose pas de cette méthode, nous allons donc calculer les **logits** puis les probabilités via la fonction **sigmoid**.

In [ ]:
X_test_tensor = torch.FloatTensor(X_test.values.astype(float))
with torch.no_grad():
    logits = model(X_test_tensor) 
    probs_pytorch = F.sigmoid(logits).numpy()[:, 1]

probabilities = {
    'Régression Logistique': lr.predict_proba(X_test)[:, 1],
    'Arbre de Décision': dt.predict_proba(X_test)[:, 1],
    'Forêt Aléatoire': rf.predict_proba(X_test)[:, 1],
    'XGBoost': xgb_model.predict_proba(X_test)[:, 1],
    'Réseau de Neurones (PyTorch)': probs_pytorch
}

plt.figure(figsize=(10, 8))

for model_name, probs in probabilities.items():
    fpr, tpr, thresholds = roc_curve(y_test, probs)
    roc_auc = roc_auc_score(y_test, probs)
    plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Taux de Faux Positifs (FPR)')
plt.ylabel('Taux de Vrais Positifs (TPR)')
plt.title('Courbes ROC de plusieurs modèles')
plt.legend(loc='lower right')
plt.grid(True)

plt.show()

Ce graphique montre que les modèles **Forêt Aléatoire**, **XGBoost** et les **Réseaux de Neurones** surpassent les autres en termes de capacité à distinguer les classes, avec des AUC proches de 1.

Pour la suite, nous allons travailler sur le modèle de **Forêt Aléatoire**, car il offre les meilleurs résultats dans l'ensemble.

## **3. Optimisation des hyperparamètres**

### **a. Grid Search pour la Forêt Aléatoire**

Le `Grid Search` est une technique visant à trouver les hyperparamètres optimaux d'un modèle en machine learning, en testant toutes les combinaisons possibles des hyperparamètres spécifiés et de déterminer celle qui donne les meilleures performances selon une métrique spécifique (comme la précision, le rappel, le F1-score).

In [ ]:
# Les paramètres à tester
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'class_weight': ['balanced', 'balanced_subsample']
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_
y_pred_best_rf = best_rf.predict(X_test)

print("Meilleurs paramètres :")
pd.DataFrame(grid_search.best_params_, index=["Paramètres"])

In [ ]:
predictions_rf = {
    'Forêt Aléatoire': y_pred_rf,
    'Forêt Aléatoire (Grid Search)': y_pred_best_rf
}
results_rf = []

for model_name, y_pred in predictions_rf.items():
    report = classification_report(y_test, y_pred, output_dict=True)
    precision_0 = report['0']['precision']
    precision_1 = report['1']['precision']
    recall_0 = report['0']['recall']
    recall_1 = report['1']['recall']
    f1_0 = report['0']['f1-score']
    f1_1 = report['1']['f1-score']

    results_rf.append({
        'Modèle': model_name,
        'Précision False': precision_0,
        'Précision True': precision_1,
        'Recall False': recall_0,
        'Recall True': recall_1,
        'F1-Score False': f1_0,
        'F1-Score True': f1_1
    })

df_results_rf = pd.DataFrame(results_rf)
df_results_rf

Les performances sont extrêments proches avec notre modèle de départ. Seule le **F1-Score** connait une réelle amélioration.

### **b. Cross Validation**

Nous allons réaliser un test de **robustesse** en utilisant la méthode de `Stratified K-Fold Cross-Validation`.

En quoi cela consiste:
- **Stratification** : Assure que chaque fold a approximativement la même proportion de chaque classe que l'ensemble de données initial.

- **K-Fold** : L'ensemble de données est divisé en K sous-ensembles de taille égale.

- **Cross Validation** : Cela consiste à entraîner et tester le modèle sur les sous-ensembles.

Les objectifs sont les suivants:
- **Estimation plus fiable** : Réduit la variance associée à une seule division entraînement/test.

- __Utilisation efficace des données__ : Toutes les observations sont utilisées à la fois pour l'entraînement et le test.

- **Détection du surapprentissage** : Permet de vérifier si le modèle généralise bien.

In [ ]:
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

scoring = {
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('scaler', StandardScaler()),  # Si nécessaire
    ('classifier', best_rf)
])

cv_results = cross_validate(
    estimator=pipeline,
    X=X,
    y=y,
    cv=skf,
    scoring=scoring,
    n_jobs=-1
)

for metric in scoring.keys():
    scores = cv_results[f'test_{metric}']
    print(f"{metric} : Moyenne = {scores.mean():.4f}, Écart-type = {scores.std():.4f}")

Les résultats en validation croisée montrent que le modèle n'est pas très stable et que ses performances sont peu robustes, surtout lorsque l'on regarde la `precision`, `recall` et `f1` sur différents sous-échantillons. Néanmoins, la performance moyenne légèrement inférieure à celle obtenue sur l'ensemble du dataset peut s'expliquer par le fait que le modèle est testé sur des portions trop petites des données lors de chaque itération.

# **IV. Analyse des résultats**

## **1. Importance des Variables**

In [ ]:
importances = best_rf.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(12, 6))
plt.title("Importance des caractéristiques")
sns.barplot(x=X_train.columns[indices][:10], y=importances[indices][:10])
plt.xticks(rotation=45)
plt.show()

`PageValues` est de loin la variable la plus explicative. Cela est tout à fait cohérent, étant que **PageValues** est une valeur basée sur la probabilité qu'une page spécifique mène à une conversion, en se référant aux données historiques du site.

On remarque que les **variables catégorielles** ne font pas partie des features les plus importantes.

## **2. Analyse des erreurs**

Nous allons garder uniquement les **10 features** les plus importantes.

In [ ]:
top_features = X_train.columns[indices][:10]

# Ajout des vraies étiquettes et des prédictions
X_test_df = X_test.copy()[top_features]
X_test_df['True_Label'] = y_test.values
X_test_df['Predicted_Label'] = y_pred_best_rf

# Instances mal classées
misclassified = X_test_df[X_test_df['True_Label'] != X_test_df['Predicted_Label']]
print(f"Nombre d'instances mal classées : {len(misclassified)} sur {len(X_test_df)} total")

# Instances bien classées
correctly_classified = X_test_df[X_test_df['True_Label'] == X_test_df['Predicted_Label']]

On peut commencer par calculer les **moyennes** et **écarts-types** pour chaque variable en fonction de si elles ont été (ou non) correctement classifiées.

In [ ]:
# Moyennes et écarts-types pour les cas mal classés
misclassified_stats = misclassified[top_features].describe().T[['mean', 'std']]

# Moyennes et écarts-types pour les cas bien classés
correctly_classified_stats = correctly_classified[top_features].describe().T[['mean', 'std']]

# Comparaison des statistiques
comparison_stats = misclassified_stats.join(correctly_classified_stats, lsuffix='_misclassified', rsuffix='_correctly_classified')
comparison_stats

Nous allons observer les **densités** de distribution de chaque features en fonction de s'ils ont été correctement classifiés ou non.

Cela nous permettra potentiellement de déterminer des caractéristiques sur les données qui ont amené à la mauvaise classification.

In [ ]:
numeric_features = X_test_df.select_dtypes(include=[np.number]).columns.tolist()
numeric_features.remove('True_Label')
numeric_features.remove('Predicted_Label')

plt.figure(figsize=(16, 12))
for i, feature in enumerate(numeric_features, 1):
    
    # Calcul de la limite pour l'axe x : 1.5 * l'écart-type autour de la moyenne
    # pour inclure 99% des données et éliminer les valeurs aberrantes
    min_val = min(comparison_stats.loc[feature, 'mean_misclassified'] - 1.5 * comparison_stats.loc[feature, 'std_misclassified'],
                  comparison_stats.loc[feature, 'mean_correctly_classified'] - 1.5 * comparison_stats.loc[feature, 'std_correctly_classified'])
    max_val = max(comparison_stats.loc[feature, 'mean_misclassified'] + 1.5 * comparison_stats.loc[feature, 'std_misclassified'],
                  comparison_stats.loc[feature, 'mean_correctly_classified'] + 1.5 * comparison_stats.loc[feature, 'std_correctly_classified'])
    
    # Tracé des distributions de densité
    plt.subplot(4, 4, i)
    sns.kdeplot(data=correctly_classified, x=feature, label='Bien classés', fill=True)
    sns.kdeplot(data=misclassified, x=feature, label='Mal classés', fill=True)
    
    # Définir la limite des axes en fonction de 1.5 * std
    plt.xlim(min_val, max_val)
    plt.title(f'Distribution de {feature}')
    plt.legend()

plt.tight_layout()
plt.show()

Si on regarde uniquement les quatres variables les plus importantes, on voit que les distributions entre `ProductRelated_Duration` et `Administrative` sont assez clairement distinctes, cependant, celles de `PageValues` et `ExitRates` ne le sont pas autant que ce dont on s'attendrait.

Nous pouvons réaliser des tests statistiques pour étudier ces distributions plus en détail. Si les `p-valeurs` sont proches de 0, cela signifie que les différences entre les distributions sont **significatives**.

In [ ]:
ks_data = []
for feature in numeric_features:
    stat, p_value = ks_2samp(correctly_classified[feature], misclassified[feature])
    ks_data.append([f'{stat:.3f}', f'{p_value:.3f}'])
pd.DataFrame(ks_data, columns=['Statistique KS', 'p-value'], index=numeric_features)

Les 8 **features** les plus importantes présentent des distribution significativement différentes. 

Cela met en lumière certains comportements de notre modèle, par exemple pour les cas mal classés, les distributions sont plus étalées avec des valeurs très basses et très élevées. Aussi, il semble y avoir une tendance où les moyennes des distributions des cas mal classés sont plus "vers la droite".

En effet, on a déjà pu l'observer sur notre DataFrame de moyennes et écart-types, mais les distributions pour les cas mal classés ont tendance a avoir un `mean`et `std` plus élevé.

Nous pourrions donc envisager de **segmenter** les données selon `PageValues` et entraîner des modèles spécifiques et ajouter des **transformations** pour mieux capturer les extrêmes. L'exploration de méthodes pour améliorer les performances de notre modèle seront explorées dans une seconde partie.

# **V. Conclusion**

Nous avons utilisé différentes méthodes de machine learning pour prédire l'intention d'achat en ligne des utilisateurs en analysant le dataset "Online Shoppers Purchasing Intention". En appliquant diverses techniques de prétraitement, telles que l'encodage des variables catégorielles, la normalisation des variables numériques et la gestion du déséquilibre des classes via le sur-échantillonnage avec `SMOTE`, nous avons entraîné plusieurs modèles de classification. Parmi ces modèles, la **forêt aléatoire** et **XGBoost** ont démontré les meilleures performances, avec des scores `F1` élevés, indiquant une bonne capacité à distinguer les sessions aboutissant à un achat de celles qui n'y aboutissent pas.

L'analyse des variables et des erreurs ont révélé que certaines *feature*, comme "`PageValues`" et "`ProductRelated_Duration`", étaient particulièrement déterminantes dans la prédiction. Cependant, le modèle a montré des limitations dans la classification correcte de certaines sessions, notamment celles présentant des comportements atypiques ou se situant aux extrêmes des distributions des variables.

# **VI. Aller Plus Loin**

Plusieurs axes d'amélioration peuvent être envisagés pour renforcer la robustesse et la précision du modèle.

## 1. Optimisation du traitement des données

- **Ingénierie des caractéristiques avancée** : La création de nouvelles variables combinant des informations existantes pourrait aider à capturer des relations non linéaires ou des interactions complexes entre les variables.

- **Transformation des variables** : L'application de transformations logarithmiques ou la normalisation robuste pourrait améliorer la gestion des valeurs extrêmes et réduire l'impact des outliers sur le modèle.

## 2. Intégration de méthodes de clusterisation

- **Segmentation des sessions** : En utilisant des algorithmes de clusterisation tels que `K-Means` ou `DBSCAN`, nous pourrions identifier des groupes homogènes de sessions basés sur le comportement des utilisateurs.
  
- **Détection d'anomalies** : La clusterisation peut également aider à identifier des sessions atypiques.

## 3. Gestion avancée du déséquilibre des classes

- **Techniques d'ensemble** : L'utilisation de méthodes comme l'`EasyEnsemble` ou le `BalancedBaggingClassifier` pourrait améliorer les performances sur la classe minoritaire en combinant les prédictions de plusieurs modèles entraînés sur des sous-échantillons équilibrés.
  
- **Pondération des classes** : Ajuster les poids associés aux erreurs de classification pour pénaliser davantage les faux négatifs pourrait améliorer le rappel.